### Scraper For BMS

In [669]:
#Importing Necessary Libraries

#importing Beautifulsoup for extract data from HTML 
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as uReq
import pandas as pd

'''
gensim is vector space modeling and topic modeling toolkit used to check the similarity of the sentences using
tf-idf which means, tf - term frequency and idf - inverse document frequency. Tf–idf is one of the most popular
term-weighting schemes today; 83% of text-based recommender systems in digital libraries use tf–idf.
'''
import gensim
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import os
import itertools
import numpy as np
from textcleaner import *

In [670]:
tc = textcleaner

In [671]:
#Defining Function Needed for striping the string and finding text similarity
# def docs_generate(y):
#     docs= y
#     return docs  

#function for shorting the name of months to 3 words
def month_shortner(x):
    h=[]
    for i in x:
        i=i[:3]
        h.append(i)
    return h

#function for checking the similarity of the words using the gensim library using dictionary and corpus of the words and tf-idf
def simila(sent):
    query_doc = [w.lower() for w in word_tokenize(sent)]
    #print(query_doc)
    query_doc_bow = dictionary.doc2bow(query_doc)
    #print(query_doc_bow)
    query_doc_tf_idf = tf_idf[query_doc_bow]
    #print(query_doc_tf_idf)
    return list(sims[query_doc_tf_idf])

# def res(k):
#     return list(k for k,_ in itertools.groupby(k))

#function for grouping the event based on the dates and checking the similarity of that events only
def eve_on_date(date):
    return tc.document(result[result.day==date]['event name'].tolist()).lower_all().remove_symbols().remove_stpwrds().token_it()

In [672]:
#stop_words = stopwords.words('english') + list(string.punctuation)

In [673]:
# Use only specified link (works only for BMS, Eventbrite and Insider as of Now)
# You can use your own link for testing other cities events on BMS, insider and eventbrite

bms_url = 'https://in.bookmyshow.com/pune/events'
eventbrite_url = 'https://www.eventbrite.com/d/india--pune/events/'
insider_url = 'https://insider.in/pune'

In [674]:
# for going to the ur and requesting and fetching the data we need from the BMS website
uClient = uReq(bms_url)
page_html = uClient.read()
uClient.close()
page_soup = soup(page_html,"html.parser")

In [675]:
containers = page_soup.findAll("div",{"class": "cards evt-card"})

In [676]:
#this is for fetching and storing the data in the python list as per need from BMS
event_name  = []
event_date =[]
event_month = []
price = []
location = []
tag = []

for container in containers:
    event_name.append(container.div.img['alt'])
    price.append(container.findAll("span",{"class": "__price"})[1].text)
    event_date.append(container.p.text)
    event_month.append(container.findAll("p",{"class":"__evt-month"})[0].text)
    location.append(container.findAll("span",{"class":"__location"})[0].text)
    #tag.append(container.findAll("span",{"class":"__tag"})[0].text)
event_month = [w.lower() for w in event_month]
event_name = [w.lower() for w in event_name]

#appending the location with the name of event for the checking similarity of both at one time
event_name = [str(w)+" at "+str(x) for w,x in zip(event_name,location)] 

In [677]:
#appending the date and month to create a day combinedly
event_day = [str(w)+" "+str(x) for w,x in zip(event_date,event_month)] 

In [678]:
#Event list of BMS and Event Date and month 
bms = list(zip(event_name,event_day))
bms

[('vh1 supersonic festival 2019 at bookmyshow at Mahalakshmi Lawns: Pune',
  '16 feb'),
 ('rambo circus at bookmyshow at Circus Ground: Pune', '1 jan'),
 ('swarazankar - a decade of musical celebration at bookmyshow at MIT - WPU Campus: Pune',
  '10 jan'),
 ('melody queen shreya ghoshal live in concert at bookmyshow at ILS Law College Ground: Pune',
  '19 jan'),
 ('lucky ali live - 30 years of musical journey at bookmyshow at Liberty Square, Phoenix Marketcity: Pune',
  '26 jan'),
 ('vipul goyal live at bookmyshow at Nomad The Lounge Bar: Aundh', '11 jan'),
 ('hot and happening by jeeveshu ahluwalia at bookmyshow at Nehru Memorial Hall: Pune',
  '10 feb'),
 ('signature masterclass 3.0 (pune) at bookmyshow at The Westin: Pune',
  '11 jan'),
 ('tribute to pink floyd by rla at bookmyshow at The High Spirits Cafe: Pune',
  '13 jan'),
 ('babuji ek sangitik parva-suresh wadkar live at bookmyshow at Ganesh Kala Krida Rangamanch: Pune',
  '26 jan'),
 ('standup comedy at viman nagar -nukkad caf

### Scraper For Eventbrite

In [679]:
# for going to the ur and requesting and fetching the data we need from the eventbrite website
uClient = uReq(eventbrite_url)
page_html2 = uClient.read()
uClient.close()
page_soup = soup(page_html2,"html.parser")

In [680]:
eventbrite_containers = page_soup.findAll("div",{"class": "eds-l-pad-all-1"})

In [681]:
event_name1  = []
event_date1 =[]
event_month1 = []
price1 = []
location1 = []

for container in eventbrite_containers:
    event_name1.append(container.div.img['alt'])
    price1.append(container.div.span.text)
    event_month1.append(container.p.text)
    event_date1.append(container.findAll("p",{"class":"date-thumbnail__day"})[0].text)
    try: 
        location1.append(container.findAll("div",{"class":"card-text--truncated__one"})[0].text) 
    except IndexError:
        location1.append('null')
#event_date1 = list(zip(event_date1,event_month1)) 
event_month1 = [w.lower() for w in event_month1]
event_name1 = [w.lower() for w in event_name1]

#appending the date and month to create a day combinedly
event_day1 = [str(w)+" "+str(x) for w,x in zip(event_date1,event_month1)] 

#appending the location with the name of event for the checking similarity of both at one time
event_name1 = [str(w)+" at "+str(x) for w,x in zip(event_name1,location1)] 

In [682]:
#Event list of eventbrite and Event Date and month
eventbrite = list(zip(event_name1,event_day1))

### Scraper For Insider

In [683]:
# for going to the ur and requesting and fetching the data we need from the Insider website
uClient = uReq(insider_url)
page_html3 = uClient.read()
uClient.close()
page_soup = soup(page_html3,"html.parser")

In [684]:
insider_containers = page_soup.findAll("div",{"class": "featured-card-details "})

In [685]:
event_name2  = []
event_date2 =[]
price2 = []
location2 = []

for container in insider_containers:
    event_name2.append(container.span.text)
    price2.append(container.findAll("span",{"class":"featured-card-price"})[0].text)
    event_date2.append(container.findAll("span",{"class":"featured-card-date"})[0].text)
    location2.append(container.findAll("span",{"class":"featured-card-venue"})[0].text)
event_name2 = [w.lower() for w in event_name2]   

#appending the location with the name of event for the checking similarity of both at one time
event_name2 = [str(w)+" at "+str(x) for w,x in zip(event_name2,location2)] 

In [686]:
k=[]
l=[]
j=[]
for e in event_date2:
     k.append(e.split())
event_month2=[]
for e in k:
    l.append(e[0])
for e in k:
    j.append(e[1])
event_date2 = j     
event_month2 = [w.lower() for w in l] 
event_month2 = month_shortner(event_month2)

#appending the date and month to create a day combinedly
event_day2 = [str(w)+" "+str(x) for w,x in zip(event_date2,event_month2)] 

In [687]:
#Event list of Insider and Event Date and month
insider = list(zip(event_name2,event_day2))

In [688]:
#Conversion of the python list to Pandas Dataframes

df_insider = pd.DataFrame(insider)
df_eventbrite = pd.DataFrame(eventbrite)
df_bms = pd.DataFrame(bms)

In [689]:
df_bms

,0,1
0,vh1 supersonic festival 2019 at bookmyshow at ...,16 feb
1,rambo circus at bookmyshow at Circus Ground: Pune,1 jan
2,swarazankar - a decade of musical celebration ...,10 jan
3,melody queen shreya ghoshal live in concert at...,19 jan
4,lucky ali live - 30 years of musical journey a...,26 jan
5,vipul goyal live at bookmyshow at Nomad The Lo...,11 jan
6,hot and happening by jeeveshu ahluwalia at boo...,10 feb
7,signature masterclass 3.0 (pune) at bookmyshow...,11 jan
8,tribute to pink floyd by rla at bookmyshow at ...,13 jan
9,babuji ek sangitik parva-suresh wadkar live at...,26 jan


In [690]:
df_eventbrite

,0,1
0,"lol speed dating pun jan 13 at The Beer Cafe, ...",13 jan
1,registration of interest: midlands engine auto...,15 jan
2,vanarts info session - pune at JW Marriott Ho...,19 jan
3,"american education event in pune, india at Rad...",15 feb
4,international students education fair - feb 20...,24 feb
5,"m:bility | india at The Hyatt Regency Pune, Pu...",26 nov
6,core power yoga at Latin Dreamers Dance Projec...,5 jan
7,karma destiny and free will at null,5 jan
8,"the new vision of life at Urja Studio Cafe, P...",6 jan
9,a walk through the maze of understanding “data...,5 jan


In [691]:
df_insider

,0,1
0,virsaa glam & gold night new year eve at The W...,31 dec
1,carlsberg presents mad'19 at Draupada Lawns & ...,31 dec
2,texas tower nye countdown 2019 at Texas Tower,31 dec
3,earth angel music festival at Venue to be anno...,11 jan
4,"spoken fest, 2019 at JioGarden, BKC, Mumbai",12 jan
5,lvc presents abhishek upmanyu ‘thoda saaf bol’...,12 jan
6,"neville shah live, pune at Thikana Baner, Pune",13 jan
7,5 star ke lolstars ft aishwarya mohanraj and k...,18 jan
8,sulafest 2019 at Sula Vineyards,2 feb
9,the other side of me – a standup comedy specia...,2 feb


In [692]:
frames = [df_bms, df_insider, df_eventbrite]

#combining the all the dataframes from different websites
result = pd.concat(frames,sort=False)

In [693]:
#Setting the column names for the newly generated dataframe with reindexing the index value
result = result.rename(columns={ 0 :'event name', 1:'day',2:'location',3:'price'})
result = result.reset_index(drop=True)

#first we have this much events
print("At Starting we have this much events : "+str(len(result)))

At Starting we have this much events : 90


In [694]:
'''
for finding the unique dates/day when events are going to happen if day was same then and then only we are going to check
the similarity of the sentences with event name as well as location combined based on tf-idf (tf increses the value of the
words as it seems more in documents and idf - inverse document frequency is set up weight for words based on its frequency
the more word seems in the document it have low weigh and the less or unique words are come in the document increses the 
weight of the words and by multiplying the value of both tf and idf we get the similarity of the sentences with another 
sentences going to happen on the same day)
'''
dates = list(set(sorted(result['day'].tolist())))
#dates

In [695]:
#checks if sentence have more than 70% similarity based on tf-idf thresold if greater then that then only it considers it as duplicate
threshold = 0.7

In [696]:
#unique_list is here for storing the unique values remaining by removing the duplicates 
unique_list = []

'''
this loop checks each date and make a unique list of the event for the events that are going to happen on same day. And
after that if on a day if there is more then one event then only we are going to check the similarity because it's obvious,
if there is one event is going to happen on the day then it will be unique for sure.
'''
for each in dates:
    #each= '8 feb'
    temp = eve_on_date(each)
    
    if len(temp)>1:
        
        #making the dictionary using the event list with location for the same day
        dictionary = gensim.corpora.Dictionary(temp)
        
        corpus = [dictionary.doc2bow(gen_doc) for gen_doc in temp]
        
        #importing the tf-idf model of the gensim and train in using the data available in our corpus of words for the same day event with location
        tf_idf = gensim.models.TfidfModel(corpus)
    
        PROJECT_ROOT = os.path.dirname(os.path.realpath('WebScraping.ipynb'))
        sims = gensim.similarities.Similarity('PROJECT_ROOT',tf_idf[corpus],num_features=len(dictionary)) 
        
        df = []
        for event in temp:
            tempa = simila(" ".join(event))
            df.append(tempa)
        #similarity matrix is created            
        df = pd.DataFrame(df)
        
        #printing the similarity for various event for various dates, You can check for better insight 
        print(df)    
        
        #sort out only values from the vectorized matrix which have similarity of more than 70%.
        k = []
        for x in range(len(df)):
            k.append(np.where(df[x]>threshold)[0].tolist())
        #print(k)    
        try:    
            results = list(set([each[0] for each in sorted(k)]))
        except IndexError:    
            continue
        results=[temp[x] for x in results]
        #print(results)
        
        #storing the name of event with location on the results and append it to unique_list for generating the final list of unique events
        for i in results:
            unique_list.append((each," ".join(i)))
    else:
        unique_list.append((each," ".join(temp[0])))


C:\Users\An Unstoppable\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
C:\Users\An Unstoppable\Anaconda3\lib\site-packages\gensim\matutils.py:718: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


     0    1
0  1.0  0.0
1  0.0  1.0
          0    1         2         3
0  1.000000  0.0  0.009086  0.009086
1  0.000000  1.0  0.000000  0.000000
2  0.009086  0.0  1.000000  1.000000
3  0.009086  0.0  1.000000  1.000000
          0         1         2    3    4
0  1.000000  0.009514  0.061515  0.0  0.0
1  0.009514  1.000000  0.010227  0.0  0.0
2  0.061515  0.010227  1.000000  0.0  0.0
3  0.000000  0.000000  0.000000  1.0  1.0
4  0.000000  0.000000  0.000000  1.0  1.0
         0        1    2         3         4         5         6         7
0  1.00000  0.06198  0.0  0.000000  0.000000  0.000000  0.000000  0.000000
1  0.06198  1.00000  0.0  0.000000  0.000000  0.000000  0.000000  0.000000
2  0.00000  0.00000  1.0  0.000000  0.000000  0.000000  0.000000  0.000000
3  0.00000  0.00000  0.0  1.000000  1.000000  0.016048  0.089773  0.089773
4  0.00000  0.00000  0.0  1.000000  1.000000  0.016048  0.089773  0.089773
5  0.00000  0.00000  0.0  0.016048  0.016048  1.000000  0.016760  0.016760
6 

In [697]:
#generation of the dataframe from Pyhton list
final_unique = pd.DataFrame(unique_list)

#renaming of the columns of the dataframe
final_unique = final_unique.rename(columns={ 0 :'day',1:'event name'})

In [698]:
print("This is the List of Final Event with date/day and location")
print(final_unique)

This is the List of Final Event with date/day and location
       day                                         event name
0   31 jan  tribute pink floyd feat rla thursday live book...
1   31 jan  study australiaattend idps free education fair...
2    5 jan  core power yoga latin dreamers dance project p...
3    5 jan                            karma destiny free null
4    5 jan  walk maze understanding data visualization sym...
5   23 jan  comedy night vandaag gateway hotel bookmyshow ...
6   10 jan  swarazankar decade musical celebration bookmys...
7   10 jan  tutc sangam nivas prayagraj bookmyshow sangam ...
8   10 jan  fiddlecraft thursday live bookmyshow hard rock...
9   10 jan  professional speaker course lonavala lonavala ...
10  12 jan  goa sunsplash 2019 bookmyshow riva beach resor...
11  12 jan              spoken fest 2019 jiogarden bkc mumbai
12  12 jan  lvc presents abhishek upmanyu thoda saaf bolin...
13  12 jan  nyu tandon meet current nyu engineering studen...
14  12 jan 

In [699]:
print("After Removing events on SImilarity basis we have remaining events : "+str(len(final_unique)))

After Removing events on SImilarity basis we have remaining events : 66
